In [ ]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.6 MB/s eta 0:00:00


In [ ]:
#3. Construct a deep learining model to find the efficient hyper parameters.

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt

#Define model building function
def build_model(hp):
    model = keras.Sequential()
    #Input Layer
    model.add(layers.Flatten(input_shape=(28, 28)))
    #Tuing the number of hidden layers and neurons
    for i in range(hp.Int("num_layers", 1, 3)):
        model.add(
            layers.Dense(
                units=hp.Int("units_" + str(i), min_value=32, max_value=512, step=32),
                activation=hp.Choice('activation_'+str(i), ['relu', 'tanh', 'sigmoid'])
            )
        )
    #Output Layer
    model.add(layers.Dense(10, activation="softmax"))

    #Compile and tuning learing rate
    model.compile(
        optimizer=keras.optimizers.Adam(
        learning_rate=hp.Choice("learning_rate", [0.01,0.001,0.0001])),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
        )
    return model


#Load mnist dataset
(x_train,y_train),(x_test,y_test) = keras.datasets.mnist.load_data()
#Normalize
x_train,x_test = x_train/255.0,x_test/255.0

#Create tuner for hyper parameter tuning
tuner = kt.Hyperband(build_model,
                     objective="val_accuracy",
                     max_epochs=10,
                     factor=3,
                     directory="my_dir",
                     project_name="intro_to_kt")
#Search for best hp
tuner.search(x_train,y_train,epochs=10,validation_split=0.2)
#get_best_hps and hypermodel
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)

#Train the model
history=model.fit(x_train,y_train,epochs=10,validation_split=0.2)

#Evaluate the model
eval_result = model.evaluate(x_test, y_test)
print(f"\nTest Accuracy: {eval_result[1]*100:.2f}%")

#Print the best hyperparameters
print("\nThe best hyperparameters are:")
for param,value in best_hps.values.items():
  print(f"{param}: {value}")


Trial 30 Complete [00h 02m 18s]
val_accuracy: 0.9766666889190674

Best val_accuracy So Far: 0.9790833592414856
Total elapsed time: 00h 25m 07s
Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.8786 - loss: 0.4203 - val_accuracy: 0.9639 - val_loss: 0.1183
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 20s 9ms/step - accuracy: 0.9732 - loss: 0.0900 - val_accuracy: 0.9726 - val_loss: 0.0862
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.9837 - loss: 0.0521 - val_accuracy: 0.9712 - val_loss: 0.0943
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.9883 - loss: 0.0372 - val_accuracy: 0.9782 - val_loss: 0.0735
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.9930 - loss: 0.0231 - val_accuracy: 0.9780 - val_loss: 0.0827
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.9936 - loss: 0.0206 - val_accuracy: 0.9781 - val_loss: 0.0809
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 21s 9ms/step - accuracy: 0.9